In [1]:
import pandas as pd
from pynif import NIFCollection
import requests
import time

In [2]:
s = ""
with open ('./training/oke17task1Training.xml.ttl') as f:
    s = f.read()
oke_task1_col = NIFCollection.loads(s, format='turtle')

for context in oke_task1_col.contexts[:2]:
    print(context.mention)
    for phrase in context.phrases:
        print(f'\t{phrase}')

Christoph Waltz also portrayed computer genius Qohen Leth in the film The Zero Theorem (2013), American plagiarist Walter Keane in the biographical film Big Eyes (2014), and 007's nemesis and head of SPECTRE Ernst Stavro Blofeld in Spectre (2015). 
	<NIFPhrase 208-228: 'Ernst Stavro Blofeld'>
	<NIFPhrase 115-127: 'Walter Keane'>
	<NIFPhrase 174-179: "007's">
	<NIFPhrase 47-57: 'Qohen Leth'>
	<NIFPhrase 200-207: 'SPECTRE'>
	<NIFPhrase 0-15: 'Christoph Waltz'>
From the largest elephant to the smallest shrew, the city of Zootopia is a mammal metropolis where various animals live and thrive. When Judy Hopps becomes the first rabbit to join the police force, she quickly learns how tough it is to enforce the law. Determined to prove herself, Judy jumps at the opportunity to solve a mysterious case. 
	<NIFPhrase 137-147: 'Judy Hopps'>
	<NIFPhrase 283-287: 'Judy'>
	<NIFPhrase 61-69: 'Zootopia'>


In [3]:
s = ""
with open ('./training/oke17task2Training.xml.ttl') as f:
    s = f.read()
    
oke_task2_col = NIFCollection.loads(s, format='turtle')

for context in oke_task2_col.contexts[:2]:
    print(context.mention)
    for phrase in context.phrases:
        print(f'\t{phrase}')

The authorities in Augsburg, southern Germany, plan to evacuate 54,000 people from the city centre on Christmas Day because of a World War Two bomb. The 3.8-tonne British bomb was found during construction work. Defusing it is expected to take most of the day. It will be Germany's biggest post-war evacuation for an unexploded bomb, Spiegel news reports. In 2011, 45,000 residents of Koblenz were evacuated. 
	<NIFPhrase 102-115: 'Christmas Day'>
	<NIFPhrase 29-45: 'southern Germany'>
	<NIFPhrase 334-341: 'Spiegel'>
	<NIFPhrase 272-281: "Germany's">
	<NIFPhrase 19-27: 'Augsburg'>
	<NIFPhrase 129-142: 'World War Two'>
	<NIFPhrase 163-170: 'British'>
	<NIFPhrase 385-392: 'Koblenz'>
Miami-born filmmaker Barry Jenkins wrote and directed the film, based on a play by Tarell Alvin McCraney, about a young African American boy named Chiron coming to terms with his sexuality as he grows up in a tough Florida neighbourhood. However, while the shoot took only 25 days, Jenkins says he conceived the pr

In [6]:
def get_scores(spots, ground_truth):
#     spots = sorted(spots, key=lambda s: s['offset'])
#     ground_truth = sorted(ground_truth, key=lambda s: s.beginIndex)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for spot in spots:
        matching = [x for x in ground_truth if x.beginIndex == spot['offset']]
        if not matching:
            false_positives += 1
            continue
        matching = matching[0]
        if spot['uri'] != matching.taIdentRef:
            false_positives += 1
        true_positives += 1
    for true_spot in ground_truth:
        matching = [x for x in spots if true_spot.beginIndex == x['offset']]
        if not matching:
            false_negatives += 1
#     print(f'tp: {true_positives} fp: {false_positives} fn: {false_negatives}')
    
    precision = 0
    recall = 0
    f1 = 0
    
    if true_positives + false_positives + false_negatives == 0:
        # Empty gold standard. Set all as 1
        precision = 1
        recall = 1
    
    if true_positives + false_positives != 0:
        precision = true_positives / (true_positives + false_positives)
    if true_positives + false_negatives != 0:
        recall = true_positives / (true_positives + false_negatives)
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    
    return {
        'tp': true_positives,
        'fp': false_positives,
        'fn': false_negatives,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [7]:
def _test_context(context, service_url, filter_policy):
    body = {
        'text': context.mention,
        'filter_policy': filter_policy
    }
    spots = requests.post(service_url, json=body).json()
    scores = get_scores(spots['data'], context.phrases)
    
    return scores

def test_collection(collection, service_url, filter_policy = 'NONE'):
    results = []
    for context in collection.contexts:
        result = _test_context(context, service_url, filter_policy)
        result['uri'] = context.uri
        results.append(result)
        time.sleep(0.5)
    return pd.DataFrame(results)

In [8]:
scores_spotlight = test_collection(oke_task1_col, 'http://localhost:4010/annotate')
print(scores_spotlight.head())

scores_spotlight2 = test_collection(oke_task2_col, 'http://localhost:4010/annotate')
print(scores_spotlight2.head())

   tp  fp  fn  precision    recall        f1  \
0   4   6   2   0.400000  0.666667  0.500000   
1   2   6   1   0.250000  0.666667  0.363636   
2   3   3   2   0.500000  0.600000  0.545455   
3   5   4   2   0.555556  0.714286  0.625000   
4   3   2   1   0.600000  0.750000  0.666667   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  precision    recall        f1  \
0   5   6   3   0.454545  0.625000  0.526316   
1   7   3   4   0.700000  0.636364  0.666667   
2   2   4   3   0.333333  0.400000  0.363636   
3  11   1   2   0.916667  0.846154  0.880000   
4  13   3   3   0.812500  0.812500  0.812500   

                                                 uri  
0  http://www.ontologydesignpatterns.

In [9]:
scores_fox = test_collection(oke_task1_col, 'http://localhost:4020/annotate')
print(scores_fox.head())

scores_fox2 = test_collection(oke_task2_col, 'http://localhost:4020/annotate')
print(scores_fox2.head())

   tp  fp  fn  precision    recall        f1  \
0   4   2   2   0.666667  0.666667  0.666667   
1   3   3   0   0.500000  1.000000  0.666667   
2   5   2   0   0.714286  1.000000  0.833333   
3   5   1   2   0.833333  0.714286  0.769231   
4   4   1   0   0.800000  1.000000  0.888889   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  precision    recall        f1  \
0   4   2   4   0.666667  0.500000  0.571429   
1   5   4   6   0.555556  0.454545  0.500000   
2   4   2   1   0.666667  0.800000  0.727273   
3  11   1   2   0.916667  0.846154  0.880000   
4   7   2   9   0.777778  0.437500  0.560000   

                                                 uri  
0  http://www.ontologydesignpatterns.

In [10]:
scores_tagme = test_collection(oke_task1_col, 'http://localhost:4030/annotate')
print(scores_tagme.head())

scores_tagme2 = test_collection(oke_task2_col, 'http://localhost:4030/annotate')
print(scores_tagme2.head())

   tp  fp  fn  precision    recall        f1  \
0   4   3   2   0.571429  0.666667  0.615385   
1   2   4   1   0.333333  0.666667  0.444444   
2   3   3   2   0.500000  0.600000  0.545455   
3   2   1   5   0.666667  0.285714  0.400000   
4   2   1   2   0.666667  0.500000  0.571429   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  precision    recall        f1  \
0   2   2   6      0.500  0.250000  0.333333   
1   5   3   6      0.625  0.454545  0.526316   
2   3   2   2      0.600  0.600000  0.600000   
3   8   2   5      0.800  0.615385  0.695652   
4   6   2  10      0.750  0.375000  0.500000   

                                                 uri  
0  http://www.ontologydesignpatterns.

In [12]:
scores_all_none = test_collection(oke_task1_col, 'http://localhost:4040/annotate')
print(scores_all_none.head())

scores_all_none2 = test_collection(oke_task2_col, 'http://localhost:4040/annotate')
print(scores_all_none2.head())

   tp  fp  fn  precision    recall        f1  \
0   5   9   1   0.357143  0.833333  0.500000   
1   3   8   0   0.272727  1.000000  0.428571   
2   5   6   0   0.454545  1.000000  0.625000   
3   6   4   1   0.600000  0.857143  0.705882   
4   4   3   0   0.571429  1.000000  0.727273   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  precision    recall        f1  \
0   6   8   2   0.428571  0.750000  0.545455   
1   9   5   2   0.642857  0.818182  0.720000   
2   5   5   0   0.500000  1.000000  0.666667   
3  12   3   1   0.800000  0.923077  0.857143   
4  13   3   3   0.812500  0.812500  0.812500   

                                                 uri  
0  http://www.ontologydesignpatterns.

In [13]:
scores_all_r_singles = test_collection(oke_task1_col, 'http://localhost:4040/annotate', filter_policy='REMOVE_SINGLE_SPOTS')
print(scores_all_r_singles.head())

scores_all_r_singles2 = test_collection(oke_task2_col, 'http://localhost:4040/annotate', filter_policy='REMOVE_SINGLE_SPOTS')
print(scores_all_r_singles2.head())

   tp  fp  fn  precision    recall        f1  \
0   4   2   2   0.666667  0.666667  0.666667   
1   2   3   1   0.400000  0.666667  0.500000   
2   4   2   1   0.666667  0.800000  0.727273   
3   5   2   2   0.714286  0.714286  0.714286   
4   3   0   1   1.000000  0.750000  0.857143   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  precision    recall        f1  \
0   3   1   5      0.750  0.375000  0.500000   
1   6   2   5      0.750  0.545455  0.631579   
2   2   3   3      0.400  0.400000  0.400000   
3  10   0   3      1.000  0.769231  0.869565   
4   7   1   9      0.875  0.437500  0.583333   

                                                 uri  
0  http://www.ontologydesignpatterns.

In [14]:
scores_spotlight.to_csv('scores/scores_spotlight_oke1.csv', index=False)
scores_fox.to_csv('scores/scores_fox_oke1.csv', index=False)
scores_tagme.to_csv('scores/scores_tagme_oke1.csv', index=False)
scores_all_none.to_csv('scores/scores_all_none_oke1.csv', index=False)
scores_all_r_singles.to_csv('scores/scores_all_r_singles_oke1.csv', index=False)

scores_spotlight2.to_csv('scores/scores_spotlight_oke2.csv', index=False)
scores_fox2.to_csv('scores/scores_fox_oke2.csv', index=False)
scores_tagme2.to_csv('scores/scores_tagme_oke2.csv', index=False)
scores_all_none2.to_csv('scores/scores_all_none_oke2.csv', index=False)
scores_all_r_singles2.to_csv('scores/scores_all_r_singles_oke2.csv', index=False)